# Instruksi
1. Pisahkan data mentah yang masih dalam satu kolom menjadi beberapa kolom berdasarkan delimiter ;.

2. Bersihkan dan ubah format kolom tanggal agar sesuai dengan format datetime

3. Lakukan pemetaan harga jual berdasarkan kode barang dengan mencocokkan data utama dengan data referensi harga.

4. Hitung total harga jual dengan mengalikan kolom jumlah terjual dan kolom harga jual, lalu simpan hasilnya dalam kolom baru bernama Total Sales.

# Inisialisiasi Projek

Pada tahapan ini dilakukan proses import library pandas sebagai library utama yang dibutuhkan untuk pengolahan data menggunakan Python. Selain itu, pada tahapan ini dilakukan juga import data dari sumber data Excel yang telah diberikan.

In [38]:
import pandas as pd

# Membaca data soruce
df = pd.read_excel('/content/Pre-Test DMT Intern (1).xlsx', header=None, sheet_name="Exercise")
pc = pd.read_excel('/content/Pre-Test DMT Intern (1).xlsx', header=None, sheet_name="Harga Jual Product")

# Melakukan split data berdasarkan delimiter ";"
df_split = df[0].str.split(';', expand=True)

# print(df_split.head())
display(df_split.head())
display(pc.head())

,0,1,2,3,4
0,Date / Tanggal,Customer Name / Nama Pelanggan,Product Code / Kode Barang,Description / Nama Barang,None
1,28-12-2022,PERSEROAN TERBATAS ALU AKSARA PRATAMA,8.00691.0250,5-SULFOSALICYLIC ACID DIHYDRATE FOR SYNTHESIS,3
2,22-12-2022,"BISI INTERNATIONAL TBK, PT",8.22343.0100,ACETAMIDE FOR SYNTHESIS,5
3,28-12-2022,"SARIFEED INDOJAYA, PT",1.00063.1000,ACETIC ACID GLACIAL 100% GR,75
4,07-12-2022,"BATARA ELOK SEMESTA TERPADU,PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,23


,0,1
0,Product Code / Kode Barang,Harga Jual
1,8.00691.0250,2300000
2,8.22343.0100,140000
3,1.00063.1000,245700
4,1.00063.2511,3450000


# Splitting Data Berdasarkan Kolom

Pada bagian ini pemrosesan bertujuan untuk merapikan struktur data dengan cara menjadikan baris pertama data sebagai nama kolom juga menyusun ulang indeks pertama ke 0

In [39]:
# Menetapkan Baris pertama pada data sebagai nama kolom
df_split.columns = df_split.iloc[0]
pc.columns = pc.iloc[0]
# Melakukan reset indeks pertama karena headernya sudah dihapus
df_split = df_split[1:].reset_index(drop=True)
pc = pc[1:].reset_index(drop=True);


display(df_split.head())
display(pc.head())

,Date / Tanggal,Customer Name / Nama Pelanggan,Product Code / Kode Barang,Description / Nama Barang,None
0,28-12-2022,PERSEROAN TERBATAS ALU AKSARA PRATAMA,8.00691.0250,5-SULFOSALICYLIC ACID DIHYDRATE FOR SYNTHESIS,3
1,22-12-2022,"BISI INTERNATIONAL TBK, PT",8.22343.0100,ACETAMIDE FOR SYNTHESIS,5
2,28-12-2022,"SARIFEED INDOJAYA, PT",1.00063.1000,ACETIC ACID GLACIAL 100% GR,75
3,07-12-2022,"BATARA ELOK SEMESTA TERPADU,PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,23
4,14-12-2022,"GELORA DJAJA, PT",1.00063.2500,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,10


,Product Code / Kode Barang,Harga Jual
0,8.00691.0250,2300000
1,8.22343.0100,140000
2,1.00063.1000,245700
3,1.00063.2511,3450000
4,1.00063.2500,109000


**Melakukan Pengecekan Data**

Pada proses pengecekan data dilakukan pencarian apakah terdapat data null pada baris data serta pengecekan format tanggal dengan ketentuan dd-mm-yy

In [40]:
# Melakukan Pengecekan Apakah ada Data yang Kosong
print(df_split.isnull().sum())
print(pc.isnull().sum())

# Melakukan Pengecekan Format Tanggal
df_split['Date / Tanggal'] = pd.to_datetime(df_split['Date / Tanggal'], format='%d-%m-%Y')
print(df_split[df_split['Date / Tanggal'].isna()])
display(df_split["Date / Tanggal"])

0
Date / Tanggal                    0
Customer Name / Nama Pelanggan    0
Product Code / Kode Barang        0
Description / Nama Barang         0
None                              0
dtype: int64
0
Product Code / Kode Barang    0
Harga Jual                    0
dtype: int64
Empty DataFrame
Columns: [Date / Tanggal, Customer Name / Nama Pelanggan, Product Code / Kode Barang, Description / Nama Barang, None]
Index: []


,Date / Tanggal
0,2022-12-28
1,2022-12-22
2,2022-12-28
3,2022-12-07
4,2022-12-14
5,2022-12-29
6,2022-12-13
7,2022-12-21
8,2022-12-27
9,2022-12-08


## Menggabungkan Dataframe
Pada bagian ini dilakukan proses penyatuan data antara DataFrame `df_split` dan `pc` menggunakan metode *left join* berdasarkan kolom **"Product Code"**. Selain itu, kolom yang sebelumnya tidak memiliki nama diubah menjadi **"Jumlah Terjual"**, serta dilakukan pemetaan harga jual dari data referensi ke data utama.


In [46]:
merged_df = pd.merge(df_split, pc, on='Product Code / Kode Barang', how='left')
merged_df.rename(columns={None: 'Jumlah Terjual'}, inplace=True)
merged_df["Jumlah Terjual"] = pd.to_numeric(merged_df["Jumlah Terjual"], errors='coerce')
display(merged_df.head())

,Date / Tanggal,Customer Name / Nama Pelanggan,Product Code / Kode Barang,Description / Nama Barang,Jumlah Terjual,Harga Jual
0,2022-12-28,PERSEROAN TERBATAS ALU AKSARA PRATAMA,8.00691.0250,5-SULFOSALICYLIC ACID DIHYDRATE FOR SYNTHESIS,3,2300000
1,2022-12-22,"BISI INTERNATIONAL TBK, PT",8.22343.0100,ACETAMIDE FOR SYNTHESIS,5,140000
2,2022-12-28,"SARIFEED INDOJAYA, PT",1.00063.1000,ACETIC ACID GLACIAL 100% GR,75,245700
3,2022-12-07,"BATARA ELOK SEMESTA TERPADU,PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,23,3450000
4,2022-12-14,"GELORA DJAJA, PT",1.00063.2500,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,10,109000


## Menghitung Total Penjualan

Pada bagiana ini dilakukan penghitungan total penjualan untuk setiap baris dengan mengalikan jumlah (kolom 'Jumlah Terjual') dengan harga satuan (kolom 'Harga Jual').


In [50]:
merged_df['Harga Jual'] = pd.to_numeric(merged_df['Harga Jual'], errors='coerce')
merged_df['Total Sales'] = merged_df["Jumlah Terjual"] * merged_df['Harga Jual']

df_sorted = merged_df.sort_values(by="Total Sales", ascending=False)
display(df_sorted)

,Date / Tanggal,Customer Name / Nama Pelanggan,Product Code / Kode Barang,Description / Nama Barang,Jumlah Terjual,Harga Jual,Total Sales
3,2022-12-07,"BATARA ELOK SEMESTA TERPADU,PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,23,3450000,79350000
17,2022-12-12,"MUARA MAS ABADI, CV",1.11117.0001,AM AMMONIUM 0.5-10 MG/L 150 X - 1 PACK,20,2370000,47400000
9,2022-12-08,"SIANTAR TOP Tbk, PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,7,3450000,24150000
2,2022-12-28,"SARIFEED INDOJAYA, PT",1.00063.1000,ACETIC ACID GLACIAL 100% GR,75,245700,18427500
16,2022-12-19,IR. HARDI PITOYO,1.11117.0001,AM AMMONIUM 0.5-10 MG/L 150 X - 1 PACK,5,2370000,11850000
19,2022-12-16,SINTA AQUA CULTURE,1.11117.0001,AM AMMONIUM 0.5-10 MG/L 150 X - 1 PACK,5,2370000,11850000
5,2022-12-29,"INDOFOOD CBP SUKSES MAKMUR Tbk, PT",1.00063.2511,ACETIC ACID GLACIAL 100% GR - 2.5 LTR,3,3450000,10350000
15,2022-12-22,VERDITA ADENIA,1.11109.0001,AM ALKALINITY 0.1-10 MMOL/L 200 X - 1 PACK,6,1239000,7434000
0,2022-12-28,PERSEROAN TERBATAS ALU AKSARA PRATAMA,8.00691.0250,5-SULFOSALICYLIC ACID DIHYDRATE FOR SYNTHESIS,3,2300000,6900000
14,2022-12-20,"TANJUNG MANGARAN, CV",1.11109.0001,AM ALKALINITY 0.1-10 MMOL/L 200 X - 1 PACK,5,1239000,6195000
